In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
import logging

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [3]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import (
    RichProgressBar,
    ModelCheckpoint,
    EarlyStopping,
)

from models.models import LitXLMRobertaModel
from models.data import AraSumDataModule

In [4]:
data_module = AraSumDataModule(
    generated_by=[
        "allam",
        "llama-batched",
        "claude",
        "openai",
        "jais-batched",
    ]
)

In [5]:
model = LitXLMRobertaModel()

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# # Add EarlyStopping and ModelCheckpoint callbacks
early_stopping_callback = EarlyStopping(
    monitor="val_loss",  # Monitor validation loss
    min_delta=0.0,  # Minimum change to qualify as improvement
    patience=3,  # Stop training after this epochs without improvement
    verbose=True,  # Print information at each validation step
    mode="min",  # Mode to minimize the monitored metric
)

In [7]:
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",  # Monitor validation loss
    dirpath="outputs/Arabic/ArabicFromAllModelsAIDetector/checkpoints",  # Directory to save checkpoints
    filename="best-checkpoint",  # Filename for the best checkpoint
    save_top_k=1,  # Save only the best checkpoint
    mode="min",  # Mode to minimize the monitored metric
)

In [8]:
# # Initialize a trainer with callbacks
trainer = pl.Trainer(
    devices=1,
    max_epochs=100,
    accelerator="auto",
    val_check_interval=0.25,
    check_val_every_n_epoch=1,
    callbacks=[RichProgressBar(), early_stopping_callback, checkpoint_callback],
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [22]:
# # Train the model
trainer.fit(model, datamodule=data_module)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


┏━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃    ┃ Name            ┃ Type                                ┃ Params ┃ Mode  ┃
┡━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0  │ val_accuracy    │ BinaryAccuracy                      │      0 │ train │
│ 1  │ test_accuracy   │ BinaryAccuracy                      │      0 │ train │
│ 2  │ train_accuracy  │ BinaryAccuracy                      │      0 │ train │
│ 3  │ xlm_roberta     │ XLMRobertaForSequenceClassification │  278 M │ eval  │
│ 4  │ fc              │ Linear                              │      3 │ train │
│ 5  │ sigmoid         │ Sigmoid                             │      0 │ train │
│ 6  │ train_precision │ BinaryPrecision                     │      0 │ train │
│ 7  │ val_precision   │ BinaryPrecision                     │      0 │ train │
│ 8  │ test_precision  │ BinaryPrecision                     │      0 │ train │
│ 9  │ train_recall    │ BinaryRecall                        │      0 │ train │
│ 10 │ val_recall      │ BinaryRecall                        │      0 │ train │
│ 11 │ test_recall     │ BinaryRecall                        │      0 │ train │
│ 12 │ train_f1        │ BinaryF1Score                       │      0 │ train │
│ 13 │ val_f1          │ BinaryF1Score                       │      0 │ train │
│ 14 │ test_f1         │ BinaryF1Score                       │      0 │ train │
└────┴─────────────────┴─────────────────────────────────────┴────────┴───────┘

Trainable params: 278 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 278 M                                                                                                
Total estimated model params size (MB): 1.1 K                                                                      
Modules in train mode: 14                                                                                          
Modules in eval mode: 230

Output()

/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/pytorch_lightning/tr
ainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` to `num_workers=254` in the `DataLoader` to improve 
performance.

...

Metric val_loss improved by 0.003 >= min_delta = 0.0. New best score: 0.042


Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.041


Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.040


Metric val_loss improved by 0.007 >= min_delta = 0.0. New best score: 0.034


Monitored metric val_loss did not improve in the last 3 records. Best score: 0.034. Signaling Trainer to stop.


In [9]:
model = model.__class__.load_from_checkpoint(
    "outputs/Arabic/ArabicFromAllModelsAIDetector/checkpoints/best-checkpoint.ckpt"
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
trainer.test(
    model,
    datamodule=data_module,
    # ckpt_path=checkpoint_callback.best_model_path,
    # ckpt_path="outputs/Arabic/LLamaArabicAIDetector/checkpoints/best-checkpoint.ckpt",
)


You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=254` in the `DataLoader` to improve performance.


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9944444298744202     │
│          test_f1          │    0.9944325089454651     │
│         test_loss         │   0.023137658834457397    │
│      test_precision       │    0.9980926513671875     │
│        test_recall        │     0.991121768951416     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.023137658834457397,
  'test_acc': 0.9944444298744202,
  'test_precision': 0.9980926513671875,
  'test_recall': 0.991121768951416,
  'test_f1': 0.9944325089454651}]